In [1]:
from pathlib import Path
from tbdynamics.calibration import run_calibration, calibrate

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:

OUT_PATH = Path.cwd() /'runs'
draws = 50000
tune = 10000

In [3]:

calibrate(OUT_PATH, draws, tune)

Multiprocess sampling (8 chains in 8 jobs)
DEMetropolisZ: [start_population_size, contact_rate, rr_infection_latent, rr_infection_recovered, progression_multiplier, seed_time, seed_num, seed_duration, smear_positive_death_rate, smear_negative_death_rate, smear_positive_self_recovery, smear_negative_self_recovery, screening_scaleup_shape, screening_inflection_time, screening_end_asymp]
